In [1]:
import numpy as np
import math as m

In [2]:
alphabet = np.array(['A','C','T','G'])

In [3]:
states = np.array([1, 2, 3])

In [4]:
#row number indicates the initial state 
#column indicates the final state
transition_matrix = np.matrix([[0.6, 0.4, 0], 
                               [0.25, 0.5, 0.25], 
                               [0.25, 0.25, 0.5]])
transition_matrix

matrix([[0.6 , 0.4 , 0.  ],
        [0.25, 0.5 , 0.25],
        [0.25, 0.25, 0.5 ]])

In [5]:
#row number indicates the state
#column goes ACTG - same order as alphabet
emission_probabilities = np.matrix([[0.4, 0, 0.3, 0.3], 
                                    [0.1, 0.4, 0.1, 0.4], 
                                    [0.4, 0.3, 0.3, 0]])
emission_probabilities

matrix([[0.4, 0. , 0.3, 0.3],
        [0.1, 0.4, 0.1, 0.4],
        [0.4, 0.3, 0.3, 0. ]])

In [6]:
sequence = np.array(['C','A','T','G','C','G','G','G','T','T','A','T','A','A','C'])

In [7]:
#generate the sequence of states neglecting the 0 values
def states_of_sequence():
    seq_of_states = np.chararray(len(sequence), itemsize = 20, unicode = True)
    for i in range(len(sequence)):
        max_val = -10000
        for j in range(len(states)):
            if result_matrix[j, i] > max_val and result_matrix[j, i] !=0:
                max_val = result_matrix[j, i]
                if j == 0:
                    seq_of_states[i] = 'Start Site signal'
                elif j == 1:
                    seq_of_states[i] = 'Exon'
                else: seq_of_states[i] = 'Intron'
    return print(seq_of_states)

In [8]:
#Recognize the max value between each value of the previous column times its transition probability.
#k is the column number of the selected cell, identify as the sequence.
#s is the row number of the selected cell, identify as the state.
def max_value(s, k):
    max_v = np.zeros(len(states))
    max_value = -100000000
    for st in range(len(states)):
        if transition_matrix[st ,s] == 0:
            max_v[st] = result_matrix[st, k-1]              
        else:
            max_v[st] = result_matrix[st, k-1] + m.log(transition_matrix[st ,s])
            
        for i in range(len(max_v)):
            if max_v[i] > max_value and max_v[i] != 0:
                max_value = max_v[i]
    return max_value

In [9]:
#Viterbi algorithm formula
#Creation of the result matrix plus sequence of states

result_matrix = np.zeros((len(states), len(sequence)))
for i in range (len(sequence)):
    for j in range(len(alphabet)):
        if sequence[i] == alphabet[j]:
            for s in range(len(states)):
                
                #init
                if i == 0:
                    if emission_probabilities[s, j] == 0:
                        result_matrix[s, i] = 0
                    else:
                        result_matrix[s, i] = m.log(1/3*emission_probabilities[s, j])
                #end init
                
                else:
                    if emission_probabilities[s, j] == 0:
                        result_matrix[s, i] = max_value(s,i)
                
                    else:    
                        result_matrix[s, i] = m.log(emission_probabilities[s, j]) + max_value(s,i)

print('States: \n')
states_of_sequence()

States: 

['Exon' 'Start Site signal' 'Intron' 'Intron' 'Start Site signal' 'Intron'
 'Intron' 'Intron' 'Intron' 'Intron' 'Intron' 'Intron' 'Intron' 'Intron'
 'Start Site signal']


### Forward algorithm to compute the probability of the sequence

In [10]:
def sum_value(s, k):
    sum_v = np.zeros(len(states))
    sum_value = 0
    for st in range(len(states)):
        sum_v[st] = result_matrix_fow[st, k-1] * transition_matrix[st ,s]
            
        for i in range(len(sum_v)):
            sum_value += sum_v[i]
    return sum_value

In [11]:
#Computing the probability of the sequence using the forward algorithm

result_matrix_fow = np.zeros((len(states), len(sequence)))
for i in range (len(sequence)):
    for j in range(len(alphabet)):
        if sequence[i] == alphabet[j]:
            for s in range(len(states)):
                #init
                if i == 0:
                    result_matrix_fow[s, i] = 1/3*emission_probabilities[s, j]  
                #end init
                
                else:
                    result_matrix_fow[s, i] = emission_probabilities[s, j]*sum_value(s,i)              


In [12]:
#Probability of the sequence
#sum of the last column times its ending transition probability to have the prob of the sequence
Ps = 0
for i in range(len(states)):
    Ps += result_matrix_fow[i, len(sequence)-1]*(1/3)
     

print('The probability of the sequence {} is: '.format(sequence),'\n', Ps)

The probability of the sequence ['C' 'A' 'T' 'G' 'C' 'G' 'G' 'G' 'T' 'T' 'A' 'T' 'A' 'A' 'C'] is:  
 4.132995015560347e-05


#### POSTERIOR PROBABILITIES

#### Backward algorithm to compute the posterior probability

In [13]:
#Backward
result_matrix_back = np.zeros((len(states), len(sequence)))
for i in range(len(states)):
    result_matrix_back[i, len(sequence)-1] = 1


for i in reversed(range(len(sequence))):
    if i > 0:
        for j in range(len(alphabet)):
            if sequence[i] == alphabet[j]:
                for s in range(len(states)):
                    sum_v = np.zeros(len(states))
                    sum_value = 0
                    for st in range(len(states)):
                        sum_v[st] = result_matrix_back[st, i] * emission_probabilities[st, j] * transition_matrix[s, st]
                        sum_value += sum_v[st]
                    result_matrix_back[s, i-1] = sum_value
                        
#print(result_matrix_back[:,len(sequence)-1])

In [14]:
print('P(π2=k|x)')
for k in range(len(states)):
    print('For state {}:'.format(k+1), result_matrix_fow[k, 1]*
                                       result_matrix_back[k, 1]/Ps)
print('Max probability is from state 3')
print('\n')

print('P(π9=k|x)')
for k in range(len(states)):
    print('For state {}:'.format(k+1),result_matrix_fow[k, 8]*
                                       result_matrix_back[k, 8]/Ps)
print('Max probability is from state 1')

P(π2=k|x)
For state 1: 1.444939357213058e-05
For state 2: 4.632013298321101e-06
For state 3: 1.5167986014460881e-05
Max probability is from state 3


P(π9=k|x)
For state 1: 0.005936928166244834
For state 2: 0.002227949998865663
For state 3: 0.002469208041688804
Max probability is from state 1
